In [2]:
from dotenv import load_dotenv

load_dotenv()
import os

from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec



c:\Users\Stephanie\miniconda3\envs\rag-app\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
print(os.environ['CONDA_DEFAULT_ENV'])

rag-app


In [11]:
#creating pinecone index, it can be a big database
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# pc.create_index(
#     name='rag', dimension= 1536, metric= 'cosine', spec=ServerlessSpec(cloud="aws", region='us-east-1')
# )

# openai outputs 1536 dimensions

In [4]:
import json
data = json.load(open("reviews.json"))
data["reviews"] 

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Physics',
  'stars': 5,
  'review': "Dr. Johnson's lectures are incredibly engaging and she explains complex concepts with great clarity. Her passion for physics is contagious!"},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 4,
  'review': 'Prof. Chen is knowledgeable and provides practical coding examples. Sometimes his pace is a bit fast, but overall a great instructor.'},
 {'professor': 'Dr. Sarah Williams',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Dr. Williams covers the material well, but her assignments can be vague. Office hours are helpful for clarification.'},
 {'professor': 'Prof. David Martinez',
  'subject': 'Literature',
  'stars': 5,
  'review': 'Prof. Martinez brings literature to life! His enthusiasm and deep understanding of the texts make every class a joy to attend.'},
 {'professor': 'Dr. Lisa Thompson',
  'subject': 'Biology',
  'stars': 2,
  'review': 'Dr. Thompson se

In [8]:
processed_data = []
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

for review in data["reviews"]:
    response = client.embeddings.create(
        input = review["review"],
        model = "text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values":embedding,
        "id": review["professor"],
        "metadata": {
            "review":review["review"],
            "subject":review["subject"],
            "stars": review["stars"]
        }
    })



In [9]:
processed_data[0]

{'values': [0.009995034,
  -0.0048510814,
  -0.017609706,
  0.03168002,
  0.016039412,
  0.010200668,
  0.002561072,
  0.050922334,
  -0.020451186,
  -0.008543137,
  0.018544404,
  0.0078078415,
  -0.03063316,
  0.010811337,
  0.003290136,
  0.014082779,
  -0.031655096,
  -0.01729814,
  0.033873446,
  0.058973197,
  0.026495565,
  -0.025137138,
  0.029935252,
  -0.004418005,
  -0.044890415,
  -0.06211378,
  0.030134656,
  0.021759763,
  0.03449658,
  -0.0057982416,
  0.081605345,
  0.0049476665,
  -0.03591732,
  -0.038609248,
  -0.02112417,
  0.04815563,
  -0.014680985,
  0.008063326,
  -0.0041095545,
  0.005735928,
  0.0096086925,
  -0.019441713,
  -0.028414812,
  0.011783423,
  0.04040387,
  0.0138584515,
  -0.008655301,
  -0.008910785,
  0.038634174,
  0.03163017,
  -0.0071909414,
  -0.003984928,
  0.014843,
  -0.003766832,
  -0.017111199,
  0.008113177,
  0.023753785,
  0.047009066,
  -0.015515982,
  -0.034222398,
  0.013908302,
  0.0072968737,
  -0.025511017,
  -0.0039101522,
  0.

In [13]:
index = pc.Index('rag')
index.upsert(#means insert into the database
    vectors = processed_data,
    namespace='ns1'
)

{'upserted_count': 25}

In [ ]:
index.describe_index_stats()